In [1]:
# %pip install altair openpyxl -q

In [1]:
#Run this once
%load_ext autoreload
%autoreload 2
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import altair as alt
import pandas as pd

from sade.configs.ve import biggan_config
from sade.datasets.loaders import get_datasets

config = biggan_config.get_config()
config.eval.experiment.train = "ibis-inlier"
config.eval.experiment.inlier = "ibis-hr-inlier"
config.eval.experiment.ood = "ibis-atypical"

[1716556977.013963] [fa79a04240ba:21808:f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


2024-05-24 13:22:58.892283: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-24 13:22:58.894528: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-24 13:22:58.929272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%%time
ibis_lr_ds, ibis_hr_ds, ibis_atyp_ds = get_datasets(config, training=False)

CPU times: user 1.81 ms, sys: 1.95 ms, total: 3.76 ms
Wall time: 29.7 ms


In [4]:
config.eval.experiment.ood = "ibis-ds-sa"
_, _, ibis_ds_sa_ds = get_datasets(config, training=False)

In [5]:
ibis_lr_ds[0], ibis_hr_ds[0], ibis_atyp_ds[0], ibis_ds_sa_ds[0]

({'image': '/DATA/Users/amahmood/braintyp/processed_v2/IBIS108372.nii.gz'},
 {'image': '/DATA/Users/amahmood/braintyp/processed_v2/IBIS130523.nii.gz'},
 {'image': '/DATA/Users/amahmood/braintyp/processed_v2/IBIS110940.nii.gz'},
 {'image': '/DATA/Users/amahmood/braintyp/processed_v2/IBIS105477.nii.gz'})

In [6]:
ibis_regex = re.compile(r".*IBIS(\d*)\.nii\.gz")
ibis_regex.match(ibis_lr_ds[0]["image"]).group(1)

'108372'

In [7]:
ibis_LR_ids = list(map(lambda x: ibis_regex.match(x["image"]).group(1), ibis_lr_ds))
ibis_HR_ids = list(map(lambda x: ibis_regex.match(x["image"]).group(1), ibis_hr_ds))
ibis_AT_ids = list(map(lambda x: ibis_regex.match(x["image"]).group(1), ibis_atyp_ds))
ibis_DS_ids = list(map(lambda x: ibis_regex.match(x["image"]).group(1), ibis_ds_sa_ds))

In [8]:
asd_dx = pd.read_csv(f"/ASD/ahsan_projects/braintypicality/dataset/DX_IBIS_Sept22.csv")
ibis_age_at_scan = pd.read_csv(f"/ASD/ahsan_projects/braintypicality/dataset/IBIS_AgeAtScanInDays_InfoNov2023.csv", usecols=lambda c: re.match( r"(.*age at scan)|(Identifiers)", c))
asd_metadata = pd.merge(asd_dx, ibis_age_at_scan, on="Identifiers")

In [9]:
col_match_regex = re.compile(r"(VSA.*)(?=.*ASD_Latest_DSMI?V|.*CandID)")

assert col_match_regex.match('VSA-CVD demographics,CandID')
assert not col_match_regex.match('V9-CVD demographics,ASD_Latest_DSMV')
assert col_match_regex.match('VSA demographics,ASD_Latest_DSMIV')

prefiltered_cols = [c for c in asd_dx.columns if col_match_regex.match(c)]

vsa_nopostfix_cols = [c for c in prefiltered_cols if "VSA " in c]
post_cvd_cols = [c for c in prefiltered_cols if "VSA-CVD " in c]
postfix_cols = [c for c in prefiltered_cols if "VSA-" in c]

In [10]:
postfix_map = {c.split(",")[0]:["Identifiers"] for c in prefiltered_cols}

for c in prefiltered_cols:
    postfix = c.split(",")[0]
    postfix_map[postfix].append(c)
    
postfix_map

{'VSA demographics': ['Identifiers',
  'VSA demographics,ASD_Latest_DSMIV',
  'VSA demographics,ASD_Latest_DSMV',
  'VSA demographics,CandID'],
 'VSA-CVD demographics': ['Identifiers',
  'VSA-CVD demographics,ASD_Latest_DSMIV',
  'VSA-CVD demographics,ASD_Latest_DSMV',
  'VSA-CVD demographics,CandID'],
 'VSA-Sleep demographics': ['Identifiers',
  'VSA-Sleep demographics,ASD_Latest_DSMIV',
  'VSA-Sleep demographics,ASD_Latest_DSMV',
  'VSA-Sleep demographics,CandID'],
 'VSA-PR-IPSC demographics': ['Identifiers',
  'VSA-PR-IPSC demographics,ASD_Latest_DSMIV',
  'VSA-PR-IPSC demographics,ASD_Latest_DSMV',
  'VSA-PR-IPSC demographics,CandID']}

In [11]:
combined_df = []

for postfix, cols in postfix_map.items():
    found = asd_dx[cols[-1]] != "."
    col_name_changer = {c: c.split(",")[1] for c in cols[1:]}
    combdf = asd_dx[cols][found].rename(columns=col_name_changer)
    combdf["VSA-Subtype"] = postfix.split(" ")[0]
    combined_df.append(combdf)

combined_df = pd.concat(combined_df, axis=0)
# dedup_idxs = combined_df["CandID"].drop_duplicates().index
# vsa_asd_dsm["CandID"] = vsa_asd_dsm["CandID"].astype(np.int64) 
vsa_asd_dsm = combined_df.drop_duplicates(subset="CandID")

In [12]:
vsa_age_at_scan = pd.merge(vsa_asd_dsm, ibis_age_at_scan, on="Identifiers")
vsa_age_at_scan

,Identifiers,ASD_Latest_DSMIV,ASD_Latest_DSMV,CandID,VSA-Subtype,V06 age at scan,V12 age at scan,V18 age at scan,V24 age at scan,V36 age at scan,...,VADOL age at scan,V03 age at scan,V15 age at scan,V09 age at scan,V12-CVD age at scan,V24-CVD age at scan,V06-CVD age at scan,V18-CVD age at scan,V09-CVD age at scan,VADOL-CVD age at scan
0,PHI0000,ASD- (VSA),ASD- (VSA),389101,VSA,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PHI0007,ASD- (VSA),ASD- (VSA),638943,VSA,0,369,0,761,0,...,5795,0,0,0,0,0,0,0,0,0
2,PHI0010,ASD+ (VSA),ASD+ (VSA),929204,VSA,190,373,0,727,0,...,0,0,0,0,0,0,0,0,0,0
3,PHI0014,ASD+ (VSA),ASD+ (VSA),809318,VSA,205,392,0,743,0,...,0,0,0,0,0,0,0,0,0,0
4,PHI0024,ASD- (VSA),ASD- (VSA),463809,VSA,183,373,0,741,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,UNC1043,ASD- (V24),ASD- (V24),142306,VSA-CVD,207,0,0,742,0,...,0,96,0,0,0,0,0,0,0,0
406,UNC1046,ASD- (V24),ASD- (V24),597924,VSA-CVD,191,377,0,0,0,...,0,90,0,0,0,0,0,0,0,0
407,UNC1047,ASD- (V24),ASD- (V24),837004,VSA-CVD,179,384,0,0,0,...,0,0,0,285,0,0,0,0,0,0
408,UNC1048,ASD- (V24),ASD- (V24),456912,VSA-CVD,210,362,0,0,0,...,0,96,0,0,0,0,0,0,0,0


### Some samples do not have both DSM 4 and 5

In [13]:
(vsa_asd_dsm['ASD_Latest_DSMIV'] != vsa_asd_dsm['ASD_Latest_DSMV']).sum()

87

In [14]:
vsa_asd_dsm_iv_and_v = vsa_asd_dsm[vsa_asd_dsm['ASD_Latest_DSMIV'] == vsa_asd_dsm['ASD_Latest_DSMV']]
vsa_asd_dsm_iv_and_v

,Identifiers,ASD_Latest_DSMIV,ASD_Latest_DSMV,CandID,VSA-Subtype
0,PHI0000,ASD- (VSA),ASD- (VSA),389101,VSA
6,PHI0007,ASD- (VSA),ASD- (VSA),638943,VSA
8,PHI0010,ASD+ (VSA),ASD+ (VSA),929204,VSA
9,PHI0014,ASD+ (VSA),ASD+ (VSA),809318,VSA
14,PHI0024,ASD- (VSA),ASD- (VSA),463809,VSA
...,...,...,...,...,...
1031,UNC1043,ASD- (V24),ASD- (V24),142306,VSA-CVD
1034,UNC1046,ASD- (V24),ASD- (V24),597924,VSA-CVD
1035,UNC1047,ASD- (V24),ASD- (V24),837004,VSA-CVD
1036,UNC1048,ASD- (V24),ASD- (V24),456912,VSA-CVD


### Will have to choose DSM 4?

In [15]:
ibis_lr_both = vsa_asd_dsm_iv_and_v[vsa_asd_dsm_iv_and_v.CandID.isin(ibis_LR_ids)]
ibis_lr_both.shape

(74, 5)

In [16]:
ibis_lr_pred = vsa_asd_dsm[vsa_asd_dsm.CandID.isin(ibis_LR_ids)]
ibis_hr_pred = vsa_asd_dsm[vsa_asd_dsm.CandID.isin(ibis_HR_ids)]

In [17]:
ibis_hr_pred[ibis_hr_pred['ASD_Latest_DSMIV'] != ibis_hr_pred['ASD_Latest_DSMV']]

,Identifiers,ASD_Latest_DSMIV,ASD_Latest_DSMV,CandID,VSA-Subtype
922,UNC0150,ASD- (V24),No DSMV ever administered,216279,VSA
5,PHI0006,ASD- (V36),No DSMV ever administered,769118,VSA-CVD
899,UNC0116,ASD- (V24),No DSMV ever administered,525057,VSA-CVD
927,UNC0158,ASD- (V24),No DSMV ever administered,895096,VSA-CVD


## Grabbing ROI Volumes

In [18]:
%%time
cerebellum_volumes_raw = pd.read_excel(f"/ASD/ahsan_projects/braintypicality/dataset/IBIS_v3.13_Cerebellum_2020May5.xlsx")
cerebellum_volumes_raw["CandID"] = cerebellum_volumes_raw["Combined_ID"].apply(lambda x: x.split("_")[0])
cerebellum_volumes_raw["Combined_Cblm_Vol"] = cerebellum_volumes_raw["RightCblm_totTiss"] + cerebellum_volumes_raw["LeftCblm_totTiss"]
cerebellum_volumes = cerebellum_volumes_raw.sort_values(by="Visit")
# cerebellum_volumes = cerebellum_volumes_raw.query("Visit=='V24' | Visit=='V36'")
cerebellum_volumes = cerebellum_volumes.drop_duplicates(subset="CandID",keep="last")
cerebellum_volumes.shape

CPU times: user 270 ms, sys: 18.9 ms, total: 289 ms
Wall time: 314 ms


(666, 14)

In [19]:
cerebellum_visit_groups = cerebellum_volumes.groupby(by="Visit")["Combined_Cblm_Vol"]
cerebellum_volumes["cerebellum_z_score"] = cerebellum_visit_groups.transform(lambda x: (x - x.mean()) / x.std())
# cerebllum_visit_means = cerebellum_visit_groups.aggregate("mean")
# cerebllum_visit_std = cerebellum_visit_groups.aggregate("std")
# for g,df in cerebellum_visit_groups:
#     print(g, df.shape, df.mean(), df.std())

In [20]:
cerebellum_volumes

,Combined_ID,DCCID,Visit,RightCblm_GM,LeftCblm_GM,RightCblm_WM,LeftCblm_WM,RightCblm_totTiss,LeftCblm_totTiss,Cerebellum_VQC,Seg Edited?,Cerebellum_ExcludeReason,CandID,Combined_Cblm_Vol,cerebellum_z_score
38,543402_V03,543402,V03,.,.,.,.,25177,23629,pass_flag,0,"Louis has listed as stx fail, but not sure why...",543402,48806,-1.243719
43,673832_V03,673832,V03,.,.,.,.,26262,27410,pass,0,.,673832,53672,-0.265291
46,702619_V03,702619,V03,.,.,.,.,24886,25693,pass,0,.,702619,50579,-0.887214
48,713586_V03,713586,V03,.,.,.,.,28373,28968,pass_flag,0,parcellation error,713586,57341,0.472451
58,891530_V03,891530,V03,.,.,.,.,25029,24708,pass_flag,0,parcellation error,891530,49737,-1.056518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,150649_V36,150649,V36,40058,38644,16277,17683,56335,56327,pass,0,.,150649,112662,-0.590317
1454,107524_V36,107524,V36,44579,45509,12227,11671,56806,57180,pass,0,.,107524,113986,-0.491263
1471,796006_V36,796006,V36,33468,34965,12077,11463,45545,46428,pass,0,.,796006,91973,-2.138141
1462,548690_V36,548690,V36,46586,43450,17820,19704,64406,63154,pass,0,.,548690,127560,0.524260


In [21]:
assert (cerebellum_volumes.Combined_Cblm_Vol.iloc[0] - 54991.36842105263) / 4973.286061108292 == cerebellum_volumes["cerebellum_z_score"] .iloc[0]

In [22]:
total_brain_volumes = pd.read_excel(f"/ASD/ahsan_projects/braintypicality/dataset/IBIS_v3.13_TotTiss_2020May4.xlsx")
total_brain_volumes["CandID"] = total_brain_volumes["Combined_ID"].apply(lambda x: x.split("_")[0])
total_brain_volumes = total_brain_volumes.sort_values(by="Visit")
total_brain_volumes = total_brain_volumes.drop_duplicates(subset="CandID",keep="last")
# total_brain_volumes = total_brain_volumes.query("Visit=='V24' | Visit=='V36'")
# total_brain_volumes = total_brain_volumes.drop_duplicates(subset="CandID",keep="first")

tbv_visit_groups = total_brain_volumes.groupby(by="Visit")["totTiss"]
total_brain_volumes["tbv_z_score"] = tbv_visit_groups.transform(lambda x: (x - x.mean()) / x.std())
total_brain_volumes

,Combined_ID,DCCID,Visit,totTiss,totTiss_VQC,Seg Edited?,totTiss_ExcludeReason,CandID,tbv_z_score
1314,891530_V03,891530,V03,581347,pass,0,.,891530,-0.554094
718,543402_V03,543402,V03,580294,pass_flag,0,"Louis has listed as stx fail, but not sure why...",543402,-0.575463
100,162820_V03,162820,V03,566928,pass,0,.,162820,-0.846706
88,152174_V03,152174,V03,562425,pass,0,.,152174,-0.938087
1398,948279_V03,948279,V03,662636,pass,0,.,948279,1.095544
...,...,...,...,...,...,...,...,...,...
414,341870_V36,341870,V36,1194146,pass,0,.,341870,0.507656
1308,886482_V36,886482,V36,1013493,pass,0,.,886482,-1.123602
1093,770655_V36,770655,V36,1274357,pass,0,.,770655,1.231944
628,490878_V36,490878,V36,1266205,pass,0,.,490878,1.158334


## Getting DAS Scores

In [23]:
# das_scores_old = pd.read_csv("/ASD/ahsan_projects/braintypicality/dataset/DAS-May82023-VSA.csv")
das_scores = pd.read_csv("/ASD/ahsan_projects/braintypicality/dataset/DAS_AllColumns_12-11-23.csv")
das_scores.shape

(404, 167)

In [24]:
regex_cvd = re.compile(r"VSA-CVD.*")
regex_non_cvd = re.compile(r"VSA\s+DAS.*")
regex_non_cvd.match('VSA DAS_SA,ABILITY_SCORE')

<re.Match object; span=(0, 24), match='VSA DAS_SA,ABILITY_SCORE'>

In [25]:
%%time
pre_cvd_cols = [c for c in das_scores.columns if regex_non_cvd.match(c)]
post_cvd_cols = [c for c in das_scores.columns if regex_cvd.match(c)]
len(pre_cvd_cols) + len(post_cvd_cols)

CPU times: user 130 µs, sys: 0 ns, total: 130 µs
Wall time: 134 µs


166

In [26]:
%%time
pre_cvd_cols = list(filter(regex_non_cvd.match, das_scores.columns))
post_cvd_cols = list(filter(regex_cvd.match, das_scores.columns))

CPU times: user 76 µs, sys: 26 µs, total: 102 µs
Wall time: 107 µs


In [27]:
das_scores_combined = das_scores[pre_cvd_cols].copy()
for pre, post in zip(pre_cvd_cols, post_cvd_cols):
    missing = das_scores_combined[pre] == "."
    das_scores_combined[pre][missing] = das_scores[post][missing]
    das_scores_combined[pre] = das_scores_combined[pre].replace(r'(^\.$)|(--)|(Spatial)', '-1', regex=True)
    das_scores_combined[pre] = das_scores_combined[pre].replace(r'(less than 0.1)|(&lt;0.1)', '0.1', regex=True)
    das_scores_combined[pre] = das_scores_combined[pre].replace(r'([Gg]reater than 99.9)', '99.9', regex=True)
    das_scores_combined[pre] = das_scores_combined[pre].replace(r'(less than 1)', '1', regex=True)
    das_scores_combined[pre] = das_scores_combined[pre].replace(r'(\*)', '1', regex=True)

score_cols = list(filter(lambda c: re.match(r".*(PERCENTILE|RAW|T_Score|ABILITY_SCORE)", c), das_scores_combined.columns))
non_score_cols = list(filter(lambda c: not re.match(r".*ABILITY_SCORE", c), das_scores_combined.columns))

# missing_all_scores = (das_scores_combined.drop(columns=["CandID"]) == "-1").all(axis=1)
das_scores_combined = das_scores_combined.drop(columns=non_score_cols)
das_scores_combined = das_scores_combined.dropna(axis=1, how="all")
# drop rows with NaNs everywhere
# das_scores_combined = das_scores_combined.dropna(axis=0, how="all")
das_scores_combined = das_scores_combined.astype(np.float32)
das_scores_combined["Identifiers"] = das_scores["Identifiers"]

/tmp/ipykernel_21808/304429951.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  das_scores_combined[pre][missing] = das_scores[post][missing]


In [28]:
score_cols

['VSA DAS_SA,ABILITY_SCORE',
 'VSA DAS_SA,GCA_PERCENTILE',
 'VSA DAS_SA,GCA_T_Score',
 'VSA DAS_SA,PC_ABILITY_SCORE',
 'VSA DAS_SA,PC_PERCENTILE',
 'VSA DAS_SA,PC_RAW',
 'VSA DAS_SA,PERCENTILE',
 'VSA DAS_SA,RAW',
 'VSA DAS_SA,SNC_PERCENTILE',
 'VSA DAS_SA,SNC_T_Score',
 'VSA DAS_SA,matrices_ABILITY_SCORE',
 'VSA DAS_SA,matrices_PERCENTILE',
 'VSA DAS_SA,matrices_RAW',
 'VSA DAS_SA,non_verb_reasoning_PERCENTILE',
 'VSA DAS_SA,non_verb_reasoning_T_Score',
 'VSA DAS_SA,rd_ABILITY_SCORE',
 'VSA DAS_SA,rd_PERCENTILE',
 'VSA DAS_SA,rd_RAW',
 'VSA DAS_SA,seq_qr_ABILITY_SCORE',
 'VSA DAS_SA,seq_qr_PERCENTILE',
 'VSA DAS_SA,seq_qr_RAW',
 'VSA DAS_SA,spatial_PERCENTILE',
 'VSA DAS_SA,spatial_T_Score',
 'VSA DAS_SA,verbal_PERCENTILE',
 'VSA DAS_SA,verbal_T_Score',
 'VSA DAS_SA,verbal_similarities_ABILITY_SCORE',
 'VSA DAS_SA,verbal_similarities_PERCENTILE',
 'VSA DAS_SA,verbal_similarities_RAW',
 'VSA DAS_SA,word_defn_ABILITY_SCORE',
 'VSA DAS_SA,word_defn_PERCENTILE',
 'VSA DAS_SA,word_defn_RAW

### ADOS Scores

In [29]:
ados_scores = pd.read_csv(f"/ASD/ahsan_projects/braintypicality/dataset/ADOS-IBIS1_2.csv")
ados_scores

,Identifiers,"V24 ADOS_Derived,ADOS_RRB_CSS","V36 ADOS_Derived,ADOS_RRB_CSS","V37Plus ADOS_Derived,ADOS_RRB_CSS","V24 ADOS_Derived,ADOS_SA_CSS","V36 ADOS_Derived,ADOS_SA_CSS","V37Plus ADOS_Derived,ADOS_SA_CSS","V24 ADOS_Derived,ADOS_classification","V36 ADOS_Derived,ADOS_classification","V37Plus ADOS_Derived,ADOS_classification",...,"V37Plus ADOS_Derived,restricted_repetitive_behavior_total","V24 ADOS_Derived,severity_score_lookup","V36 ADOS_Derived,severity_score_lookup","V37Plus ADOS_Derived,severity_score_lookup","V24 ADOS_Derived,social_affect_restricted_repetitive_behavior_total","V36 ADOS_Derived,social_affect_restricted_repetitive_behavior_total","V37Plus ADOS_Derived,social_affect_restricted_repetitive_behavior_total","V24 ADOS_Derived,social_affect_total","V36 ADOS_Derived,social_affect_total","V37Plus ADOS_Derived,social_affect_total"
0,PHI0000,NaN,NaN,.,NaN,NaN,.,autism,none,.,...,.,8,1,.,17,2,.,14,1,.
1,PHI0002,NaN,NaN,.,NaN,NaN,.,autism,autism,.,...,.,8,10,.,17,20,.,15,14,.
2,PHI0003,NaN,NaN,.,NaN,NaN,.,autism,autism,.,...,.,8,10,.,17,26,.,14,20,.
3,PHI0005,NaN,NaN,.,NaN,NaN,.,autism,autism,.,...,.,7,6,.,21,14,.,18,10,.
4,PHI0006,NaN,NaN,.,NaN,NaN,.,none,none,.,...,.,1,1,.,3,2,.,3,1,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,UNC1041,NaN,.,.,NaN,.,.,none,.,.,...,.,1,.,.,1,.,.,1,.,.
616,UNC1043,NaN,.,.,NaN,.,.,none,.,.,...,.,2,.,.,4,.,.,4,.,.
617,UNC1046,NaN,.,.,NaN,.,.,none,.,.,...,.,1,.,.,2,.,.,1,.,.
618,UNC1047,NaN,.,.,NaN,.,.,none,.,.,...,.,1,.,.,2,.,.,0,.,.


In [30]:
ados_scores = pd.read_csv(f"/ASD/ahsan_projects/braintypicality/dataset/ADOS-IBIS1_2.csv",
                 usecols=lambda c: re.match( r"(.*_total)|(.*severity_score_lookup)|(Identifiers)", c))
ados_scores = ados_scores.replace(".", -1).dropna()

In [31]:
cols = [c for c in ados_scores if c != "Identifiers"]
ados_scores[cols] = ados_scores[cols].astype(np.float32)
ados_scores = ados_scores[~(ados_scores[cols] == -1).all(axis=1)]
ados_scores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 616 entries, 0 to 619
Data columns (total 13 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   Identifiers                                                              616 non-null    object 
 1   V24 ADOS_Derived,restricted_repetitive_behavior_total                    616 non-null    float32
 2   V36 ADOS_Derived,restricted_repetitive_behavior_total                    616 non-null    float32
 3   V37Plus ADOS_Derived,restricted_repetitive_behavior_total                616 non-null    float32
 4   V24 ADOS_Derived,severity_score_lookup                                   616 non-null    float32
 5   V36 ADOS_Derived,severity_score_lookup                                   616 non-null    float32
 6   V37Plus ADOS_Derived,severity_score_lookup                               6

## Demographics

In [32]:
vsa_info = pd.read_csv(f"/ASD/ahsan_projects/braintypicality/dataset/VSADemographicsMar2023.csv")
vsa_info.rename(columns={c:c.split(",")[1] for c in vsa_info.columns if "VSA" in c}, inplace=True)
# vsa_info['CandID'] = vsa_info['CandID'].astype(np.int64)
vsa_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CandID              423 non-null    int64  
 1   Identifiers         423 non-null    object 
 2   ASD_Ever_DSMIV      423 non-null    object 
 3   ASD_Ever_DSMV       423 non-null    object 
 4   Age_at_visit_start  409 non-null    float64
 5   Risk                358 non-null    object 
 6   Sex                 423 non-null    object 
 7   candidate_race      232 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 26.6+ KB


## Updated CBCL and Vineland Scores

In [33]:
behavior_scores = pd.read_csv("/ASD/ahsan_projects/braintypicality/dataset/IBIS-SA-CBCL-Vineland-SRS2-All.csv")
behavior_scores

,Identifiers,"VSA CBCL_6_To_18,T_score","VSA-CVD CBCL_6_To_18,T_score","VSA CBCL_6_To_18,aggr_behav_T_score","VSA-CVD CBCL_6_To_18,aggr_behav_T_score","VSA CBCL_6_To_18,aggr_behav_percentile","VSA-CVD CBCL_6_To_18,aggr_behav_percentile","VSA CBCL_6_To_18,aggr_behav_score","VSA-CVD CBCL_6_To_18,aggr_behav_score","VSA CBCL_6_To_18,anx_prob_T_score",...,"VSA Vineland_SA,socialization_STD_SCORE","VSA-CVD Vineland_SA,socialization_STD_SCORE","VSA Vineland_SA,written_RAW_SCORE","VSA-CVD Vineland_SA,written_RAW_SCORE","VSA Vineland_SA,written_percent_estimated","VSA-CVD Vineland_SA,written_percent_estimated","VSA Vineland_SA,written_v_Scale_Score","VSA-CVD Vineland_SA,written_v_Scale_Score","VSA demographics,CandID","VSA-CVD demographics,CandID"
0,PHI0000,.,.,.,.,.,.,.,.,.,...,88,.,64,.,0.0,.,15,.,389101,.
1,PHI0006,.,NaN,.,50,.,50,.,0,.,...,.,108,.,74,.,0.0,.,17,.,769118
2,PHI0007,NaN,.,70,.,98,.,17,.,73,...,98,.,65,.,0.0,.,15,.,638943,.
3,PHI0010,NaN,.,50,.,50,.,1,.,70,...,92,.,59,.,0.0,.,13,.,929204,.
4,PHI0014,NaN,.,61,.,87,.,9,.,67,...,73,.,63,.,0.0,.,15,.,809318,.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,UNC0251,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,700429,.
441,UNC0252,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,717010,.
442,UNC0253,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,215153,.
443,UNC0254,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,865089,.


### Pre Covid and covid subsets are distinct, so we will merge them

In [34]:
behaviour_tests = {}

for test_name in ["CBCL", "Vineland"]:
    regex_cvd = re.compile(f"VSA-CVD\s+{test_name}.*")
    regex_non_cvd = re.compile(f"VSA\s+{test_name}.*")
    pre_cvd_cols = list(filter(regex_non_cvd.match, behavior_scores.columns))
    post_cvd_cols = list(filter(regex_cvd.match, behavior_scores.columns))
    
    col_name_changer = lambda c: f"{test_name}-{c.split(',')[1]}" if "VSA" in c else c


    precvd_df = behavior_scores[pre_cvd_cols+["Identifiers"]]
    missing = (precvd_df[pre_cvd_cols] == ".").all(axis=1)
    precvd_df = precvd_df[~missing]
    precvd_df = precvd_df.rename(columns=col_name_changer)

    cvd_df = behavior_scores[post_cvd_cols+["Identifiers"]]
    missing = (cvd_df[post_cvd_cols] == ".").all(axis=1)
    cvd_df = cvd_df[~missing]
    cvd_df = cvd_df.rename(columns=col_name_changer)

    combined_scores = pd.concat([precvd_df, cvd_df], axis=0)
    combined_scores = combined_scores.drop_duplicates(subset="Identifiers")
    combined_scores = combined_scores.dropna(axis='columns', how='all')
    
    behaviour_tests[test_name] = combined_scores

In [35]:
cbcl_scores = behaviour_tests['CBCL']
vineland_scores = behaviour_tests['Vineland']

### Combining all metadata

In [43]:
cbcl_scores.shape, vineland_scores.shape, das_scores_combined.shape, vsa_asd_dsm.shape, ados_scores.shape, vsa_info.shape

((340, 61), (364, 44), (404, 7), (422, 5), (616, 13), (423, 8))

In [44]:
ibis_metadata = pd.merge(ados_scores, vsa_asd_dsm, on="Identifiers", how="outer")
ibis_metadata = pd.merge(ibis_metadata, das_scores_combined, on="Identifiers", how="outer")
ibis_metadata = pd.merge(ibis_metadata, vineland_scores, on="Identifiers", how="outer")
ibis_metadata = pd.merge(ibis_metadata, cbcl_scores, on="Identifiers", how="outer")
# ibis_metadata = pd.merge(ibis_metadata, cerebellum_volumes[["CandID", "cerebellum_z_score", "Visit"]], on="CandID", how="outer")
# ibis_metadata = pd.merge(ibis_metadata, total_brain_volumes[["CandID", "tbv_z_score"]], on=["CandID"], how="outer")
ibis_metadata = pd.merge(ibis_metadata, vsa_info[["Identifiers", "Sex"]], on=["Identifiers"], how="outer")

# ibis_metadata = ibis_metadata.replace(np.nan, -1)
ibis_metadata = ibis_metadata.drop_duplicates(subset="CandID")
ibis_metadata = ibis_metadata.fillna(-1)
# ibis_metadata = ibis_metadata.astype(np.float32, errors="ignore")

In [46]:
ibis_metadata.Sex.value_counts()

Male      246
Female    162
-1         15
Name: Sex, dtype: int64

In [39]:
ibis_metadata.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 695
Columns: 127 entries, Identifiers to Sex
dtypes: float32(18), object(109)
memory usage: 393.3+ KB


In [47]:
ibis_metadata.to_csv("/ASD/ahsan_projects/braintypicality/dataset/ibis_metadata_v2.csv", index=None)